In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import os
from pandas import DataFrame
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import math
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.layers import LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet import ResNet101
import cv2
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
train_path = '/content/gdrive/My Drive/Colab Notebooks/Skin Cancer ISIC/train'
test_path = '/content/gdrive/My Drive/Colab Notebooks/Skin Cancer ISIC/test'

In [ ]:
Width = 224
Height = 224
Batch_Size = 100 #changed to 100 from 64 (64 gave 99,70 acc)
#batch size=100 gave test acc=77, train acc=82

train_data_generator= ImageDataGenerator(shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip=True, 
                                         preprocessing_function=preprocess_input 
                                         )

train_set = train_data_generator.flow_from_directory(train_path,
                                                    target_size=(Width,Height),
                                                    batch_size=Batch_Size,
                                                    class_mode='binary',
                                                    classes=['benign', 'malignant']                                                  
                                                  )

test_data_generator= ImageDataGenerator()

test_set = test_data_generator.flow_from_directory(test_path,
                                                  target_size=(Width,Height),
                                                  batch_size=Batch_Size, class_mode='binary', 
                                                   shuffle=False,
                                                   classes = ['benign', 'malignant']
                                                    )

# To print number of batches per epoch
print('Number of Batches per epoch train=%d,test=%d' % (len(train_set), len(test_set)))

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.
Number of Batches per epoch train=27,test=7


In [ ]:
base_model_042= ResNet50(include_top=False, weights='imagenet', input_shape=(Width, Height,3))
base_model_042.summary() 

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
#changed lr to 0.00001

In [ ]:
# Adding custom layers
x = base_model_042.output
# gap= sum of all elements/number of all (which ia a 1*1)
x = GlobalAveragePooling2D()(x) # To reduce the Number of parameters that feed to future FC layer which reduce the total weights and can improve training speed
# x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(rate=0.3)(x) # Regularization method to minimize overfitting             
#x = Dense(256, activation='relu')(x)
#x = Dropout(0.3)(x) # Regularization method to minimize overfitting
predictions = Dense(2, activation='softmax')(x) 

# creating the final model 
model_final_042 = Model(inputs = base_model_042.input, outputs = predictions)

# Freezing first seven layers
for layer in model_final_042.layers[:-40]:
    layer.trainable = False

model_final_042.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizers.Adam(lr=0.00001), metrics=["accuracy"])
model_final_042.summary()
# print ('model output shape:', model_final.output_shape)


Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
checkpoint = ModelCheckpoint("Skin_cancer6.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')

import math
# Train the model
train_042= model_final_042.fit_generator(
                                train_set,
                                steps_per_epoch = train_set.samples//train_set.batch_size,
                                epochs = 50,
                                validation_data = test_set,
                                validation_steps = test_set.samples//test_set.batch_size,
                                callbacks=[checkpoint,early])

Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 0.6348 - accuracy: 0.6488
Epoch 00001: val_accuracy improved from -inf to 0.70333, saving model to Skin_cancer6.h5
26/26 [==============================] - 37s 1s/step - loss: 0.6348 - accuracy: 0.6488 - val_loss: 0.5806 - val_accuracy: 0.7033
Epoch 2/50
26/26 [==============================] - ETA: 0s - loss: 0.4170 - accuracy: 0.8061
Epoch 00002: val_accuracy improved from 0.70333 to 0.76500, saving model to Skin_cancer6.h5
26/26 [==============================] - 38s 1s/step - loss: 0.4170 - accuracy: 0.8061 - val_loss: 0.4961 - val_accuracy: 0.7650
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 0.3570 - accuracy: 0.8356
Epoch 00003: val_accuracy improved from 0.76500 to 0.77000, saving model to Skin_cancer6.h5
26/26 [==============================] - 37s 1s/step - loss: 0.3570 - accuracy: 0.8356 - val_loss: 0.4943 - val_accuracy: 0.7700
Epoch 4/50
26/26 [==============================] - ETA: 0

In [ ]:
test = '/content/gdrive/My Drive/Colab Notebooks/Skin Cancer ISIC/train/malignant/1010.jpg'

In [ ]:
img = image.load_img(test, target_size=(224,224)) 
img = image.img_to_array(img)
img = np.array([img])

In [ ]:
pred = model_final_042.predict(img)
pred.argmax()

0